# NFL Pandas on Spark
Chien-Lan Hsueh

## NFL
To illustrate the ease of parallelization with Spark, we can now update an earlier MapReduce algorithm to run on Spark with far fewer steps required. Previously, we had to read in the large scores dataset, subset based on the value of season, then reread these datasets as an iterable list for the map and reduce functions. Now, we can simply use a pandas on spark dataframe to iterate over the values of seasons and parallelize our earlier computations.

Starting Spark session

### Preparation
- Load modules
- Set up configuration to run Spark
- Define aggregation function to work on data (pyspark.pandas.DataFrame)
- Read in NFL data (with a quick inspection)

In [1]:
# load needed modules and configure environment
import os
import sys
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

import pandas as pd
import numpy as np
import matplotlib
import plotly
plotly.io.renderers.default = "notebook_connected"

import pyspark.pandas as ps
from pyspark.sql import SparkSession

In [2]:
# a helper function for easy aggregation
def aggPSDF(df, by = ["All"], values = None, func = ["mean", "std"]):
    """
    Aggregat using one or more operations on the specified columns
    df: data
    by: list of columns used to determine the groups for the groupby
    values: list of columns to aggregate
    aggfunc: list of aggregation functions
    """    
    return df[by + vals].groupby(by).agg(func).sort_index() 

The scores dataset contains various statistics and information on football teams' performance from the 2002 to 2014 seasons. We're primarily interested in summarizing the quarterly and final scores for the Away and Home Teams, on average.

Reading in full scores dataset to a pandas-on-Spark dataframe so Spark can parallelize the operations

In [3]:
# read in data and add a dummy column for aggregation on whole data set
scores_full = ps.read_csv("scoresFull.csv").assign(All = "All")

In [4]:
# inspect data
scores_full.info()

<class 'pyspark.pandas.frame.DataFrame'>
Int64Index: 3471 entries, 0 to 3470
Data columns (total 83 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   week             3471 non-null   object 
 1   date             3471 non-null   object 
 2   day              3471 non-null   object 
 3   season           3471 non-null   int32  
 4   awayTeam         3471 non-null   object 
 5   AQ1              3471 non-null   int32  
 6   AQ2              3471 non-null   int32  
 7   AQ3              3471 non-null   int32  
 8   AQ4              3471 non-null   int32  
 9   AOT              3471 non-null   int32  
 10  AOT2             3471 non-null   int32  
 11  AFinal           3471 non-null   int32  
 12  homeTeam         3471 non-null   object 
 13  HQ1              3471 non-null   int32  
 14  HQ2              3471 non-null   int32  
 15  HQ3              3471 non-null   int32  
 16  HQ4              3471 non-null   int32  
 17  HOT        

### Mean and Standard Deviation of Aggregated Data
We start by select the variables (scores) of interest:

In [5]:
# variables (columns) of interest for data aggregation
vals = ['AQ1', 'AQ2', 'AQ3', 'AQ4', 'AFinal', 'HQ1', 'HQ2', 'HQ3', 'HQ4', 'HFinal']

#### (1) Mean and standard deviation of the selected scores - the whole data set

In [6]:
# on whole data set
df1 = aggPSDF(scores_full, values = vals)
display(df1)

AQ1               AQ2                AQ3                 AQ4               AFinal                  HQ1                 HQ2                 HQ3                 HQ4              HFinal           
         mean     std      mean       std     mean       std      mean       std       mean        std      mean       std      mean       std      mean       std      mean      std       mean        std
All                                                                                                                                                                                                        
All  3.924806  4.4907  6.241429  5.221593  4.38692  4.632717  5.890233  5.278775  20.557188  10.195586  4.828868  4.726903  7.105157  5.702788  4.791126  4.755145  6.322962  5.41731  23.174013  10.405952

#### (2) Mean and standard deviation of the selected scores - by `season`

In [7]:
# aggregate by season
df2 = aggPSDF(scores_full, values = vals, by = ["season"])
display(df2)

AQ1                 AQ2                 AQ3                 AQ4               AFinal                  HQ1                 HQ2                 HQ3                 HQ4               HFinal           
            mean       std      mean       std      mean       std      mean       std       mean        std      mean       std      mean       std      mean       std      mean       std       mean        std
season                                                                                                                                                                                                            
2002    4.037453  4.470297  6.022472  5.167810  4.449438  4.639673  5.928839  5.282133  20.640449  10.296996  4.307116  4.549980  7.411985  5.923282  4.715356  4.716938  6.426966  5.384231  23.018727  10.295065
2003    3.558052  4.320719  6.093633  5.440954  3.913858  4.396264  5.355805  5.148618  19.112360  10.211049  5.026217  4.681919  6.625468  5.367875  4.561798  4.719871  6.295880  5.321832  22.677903  10.105887
2004    3.913858  4.556669  6.265918  5.094638  4.153558  4.631822  5.797753  5.400181  20.239700  10.162710  4.943820  4.733562  7.071161  5.587193  4.250936  4.675238  6.561798  5.155379  22.906367  10.441145
2005    3.887640  4.522554  5.543071  4.947945  4.029963  4.476236  5.191011  4.753811  18.786517   9.926578  4.393258  4.586556  7.426966  5.704940  4.677903  4.489042  5.737828  5.173548  22.314607   9.772905
2006    3.629213  3.986015  6.179775  5.157435  4.303371  4.531605  6.063670  5.539984  20.254682  10.269065  4.606742  4.999138  6.041199  5.199966  4.565543  4.931715  5.932584  5.262574  21.258427   9.876525
2007    3.696629  4.324448  6.220974  5.050111  4.337079  4.507997  5.872659  5.283565  20.228464  10.572177  5.026217  4.824287  7.074906  5.780607  4.913858  4.455721  6.018727  6.010300  23.157303  10.500518
2008    3.801498  4.203686  6.498127  5.337747  4.074906  4.242863  6.423221  5.498864  20.842697  10.279806  5.179775  4.857120  7.205993  5.825685  4.595506  4.700950  6.044944  5.171299  23.183521  10.414349
2009    3.868914  4.748728  6.202247  5.285488  4.299625  4.714412  5.943820  5.571192  20.382022  10.743600  4.737828  4.731771  7.790262  6.208245  4.228464  4.723587  5.887640  5.176810  22.779026  10.788110
2010    3.973783  4.727466  6.865169  5.396722  4.629213  4.784959  5.704120  4.953022  21.318352  10.278809  4.576779  4.485221  6.771536  5.476904  4.868914  4.753476  6.681648  5.626849  23.000000  10.230060
2011    3.857678  4.455541  5.981273  5.281869  4.584270  4.628635  5.996255  5.120722  20.509363   9.647690  5.022472  4.859633  7.333333  5.673886  5.033708  4.962527  6.479401  5.262521  23.981273  10.527962
2012    4.468165  4.754293  6.273408  5.452819  4.509363  4.627771  6.235955  5.338234  21.655431  10.367006  4.928839  4.504654  7.003745  5.379925  5.217228  4.542779  6.823970  5.675672  24.119850  10.559919
2013    4.044944  4.345709  6.670412  5.441761  5.011236  4.904719  6.183521  5.417879  21.977528   9.680199  5.112360  4.791373  7.445693  5.726854  5.258427  4.846194  6.932584  5.757240  24.891386  10.582623
2014    4.284644  4.893746  6.322097  4.769223  4.734082  5.059837  5.876404  5.243299  21.295880   9.780606  4.913858  4.821248  7.164794  6.113817  5.397004  5.184534  6.374532  5.329217  23.973783  10.870879

#### (2) Mean and standard deviation of the selected scores - by `homeTeam` and `season`

In [8]:
# aggregate by homeTeam and season
df3 = aggPSDF(scores_full, values = vals, by = ["homeTeam", "season"])
display(df3.head(26))

AQ1                  AQ2                 AQ3                 AQ4               AFinal                  HQ1                  HQ2                 HQ3                 HQ4               HFinal           
                              mean       std       mean       std      mean       std      mean       std       mean        std      mean       std       mean       std      mean       std      mean       std       mean        std
homeTeam          season                                                                                                                                                                                                              
Arizona Cardinals 2002    7.375000  5.396758   6.750000  6.408699  4.125000  5.938675  2.750000  4.301163  21.000000  11.426786  2.500000  3.070598   6.000000  4.174754  0.375000  1.060660  5.625000  4.627171  15.250000   5.897942
                  2003    5.500000  4.566962   6.250000  6.584614  3.875000  3.482097  5.375000  3.662064  21.375000   9.164177  4.750000  3.240370   4.000000  3.070598  2.500000  3.070598  5.000000  4.869732  16.625000   7.558108
                  2004    2.625000  3.622844   5.500000  4.276180  5.125000  3.681518  1.625000  3.020761  15.250000   8.293715  3.375000  4.068608   5.625000  2.503569  4.125000  3.044316  7.125000  7.298483  20.250000  10.898886
                  2005    5.500000  4.928054   4.625000  4.627171  4.750000  5.522681  5.125000  6.174545  20.000000   7.131419  1.500000  2.267787   7.750000  5.599745  4.750000  2.659216  5.875000  5.409978  19.875000   6.468329
                  2006    3.375000  4.068608   6.625000  4.657943  5.500000  3.207135  7.625000  5.730557  23.125000   8.025629  8.750000  8.154753   4.750000  3.615443  2.875000  2.900123  4.250000  3.991061  20.625000   7.520211
                  2007    4.625000  2.722263   4.625000  3.662064  6.000000  4.659859  7.000000  5.318432  23.000000   6.866066  4.625000  4.627171  10.875000  7.754031  3.875000  3.482097  7.875000  5.591767  27.625000  10.822430
                  2008    3.727273  6.357530   9.727273  5.746145  4.181818  3.995452  5.454545  4.274661  23.090909   8.607608  5.636364  5.065032   8.181818  6.013621  8.272727  5.255300  7.090909  4.867331  29.727273   6.813089
                  2009    4.555556  4.901814   9.333333  7.905694  4.666667  4.949747  7.111111  6.546840  25.666667  10.793517  4.555556  5.681354   7.555556  6.635343  6.111111  4.196559  6.111111  3.919325  25.000000  13.360389
                  2010    7.125000  4.998214   7.500000  5.099020  4.500000  4.242641  6.125000  2.948971  25.250000   8.481071  6.250000  4.399675   6.250000  4.166190  3.375000  4.955156  7.750000  8.730079  23.625000  13.146836
                  2011    3.375000  3.248626   7.250000  3.011881  5.500000  2.927700  4.625000  7.614601  20.750000   7.265378  2.875000  2.900123   3.375000  3.248626  6.000000  4.174754  7.625000  3.204350  22.125000   3.563205
                  2012    3.250000  3.845220   7.250000  6.430952  6.750000  4.200340  1.750000  2.866058  19.375000   8.551316  3.250000  3.615443   7.750000  7.206148  2.375000  2.386719  6.000000  5.606119  19.750000  10.361329
                  2013    5.375000  5.235524   5.000000  5.529144  2.625000  5.208167  4.750000  3.284161  17.750000   9.315885  2.625000  3.622844  10.625000  6.479363  5.375000  3.159453  8.000000  2.976095  26.625000   6.323143
                  2014    3.875000  3.482097   6.750000  4.862392  3.000000  5.656854  3.875000  7.356969  17.500000   8.349508  3.375000  4.955156   4.875000  3.181981  4.750000  5.391793  7.375000  7.818248  20.375000   8.365533
Atlanta Falcons   2002    1.250000  2.549510   2.500000  3.070598  4.750000  4.949747  5.250000  6.649382  14.500000   9.456668  4.875000  5.488625   7.250000  4.334249  5.375000  4.240536  5.375000  5.998512  22.875000   9.760525
                  2003    4.250000  5.470701  10.625000  7.670677  9.000000  4.869732  5.750000  5.03558

### Plot the means of the selected scores by season
Because [currently pyspark.pandas.DataFrame.xs() doesn't support many data manipulation on `axis = 1` (columns)](https://spark.apache.org/docs/latest/api/python/reference/pyspark.pandas/api/pyspark.pandas.DataFrame.xs.html), we have to do axis swap before we can call `DataFrame.xs()` to get cross-section for the means from the DataFrame, and then swap it back to have `season` as index for plotting:

In [9]:
# get means of each variables by season for plotting
df = df2.drop(columns = ["AFinal", "HFinal"]).swapaxes(i=1, j=0).xs("mean", level = 1).swapaxes(i=1, j=0)
display(df)

,AQ1,AQ2,AQ3,AQ4,HQ1,HQ2,HQ3,HQ4
season,,,,,,,,
2002,4.037453,6.022472,4.449438,5.928839,4.307116,7.411985,4.715356,6.426966
2003,3.558052,6.093633,3.913858,5.355805,5.026217,6.625468,4.561798,6.295880
2004,3.913858,6.265918,4.153558,5.797753,4.943820,7.071161,4.250936,6.561798
2005,3.887640,5.543071,4.029963,5.191011,4.393258,7.426966,4.677903,5.737828
2006,3.629213,6.179775,4.303371,6.063670,4.606742,6.041199,4.565543,5.932584
2007,3.696629,6.220974,4.337079,5.872659,5.026217,7.074906,4.913858,6.018727
2008,3.801498,6.498127,4.074906,6.423221,5.179775,7.205993,4.595506,6.044944
2009,3.868914,6.202247,4.299625,5.943820,4.737828,7.790262,4.228464,5.887640
2010,3.973783,6.865169,4.629213,5.704120,4.576779,6.771536,4.868914,6.681648


After carrying out the analysis by season, we can see that the Home team still does perform a bit better on average. There is still a considerable amount of variation, though, as the standard deviations are roughly the same size as the means for the quarterly statistics. To get a better idea of any trends, we visualize these averages.

In [10]:
# plot the means
pt = df.plot(title = "Average Quarterly Scores by Season")

# add titles and legend
pt.update_layout(
    title = "Average Quarterly Scores by Season",
    xaxis_title = "Season",
    yaxis_title = "Quarterly Scores",
    legend_title = "Quarterly Scores"
)

pt.show()

From the plot, we get a clearer picture of our data. It seems the Home team's second quarter scores are generally the highest of all, in variation as well as values. This trend is also seen for the Away team, their highest respective scores fall in the second quarter (though in the 2011 and 2012 seasons the fourth quarter scores are very close) with a great deal of variation across seasons compared to other quarters. This graph also depicts the higher scores for the Home team noted previously. For the quarters where the teams score the highest (second and fourth respectively), the Home team generally performs better. Likewise, where the teams score the lowest, the Away team generally scores lower.